### CNN으로 패션아이템 구분
- Convolutional Neural Network (CNN) 을 이용하여 패션아이템 구분 성능 고도화

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

from tqdm.autonotebook import tqdm # 프로그래스바 시각화 모듈

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

EPOCHS     = 40
BATCH_SIZE = 64

#### 데이터셋 로드

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./data/',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./data/',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

#### 뉴럴넷으로 Fashion MNIST

In [7]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

#### 하이퍼파라미터

In [8]:
model     = Network().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

#### 학습

In [ ]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ' + \
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

#### 테스트

In [10]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

#### 코드 실행

In [ ]:
for epoch in tqdm( range(1, EPOCHS + 1) ):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')

  0%|          | 0/40 [00:00<?, ?it/s]

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313828
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.689898
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.069033
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.230990
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.718220
[1] Test Loss: 0.7136, Accuracy: 72.44%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.867169
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.902843
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.850940
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.723771
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.793350
[2] Test Loss: 0.5926, Accuracy: 75.60%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.702417
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.688312
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.850773
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.846347
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.550859
[3] Test Loss: 0.5408, Accuracy: 79.26%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.738840
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.716539
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.762709


### 신경망 깊게 쌓아 컬러 데이터셋에 적용 - ResNet
- 잔차 학습(Residual Learning) 
    - 일반적인 깊은 신경망에서는 층이 깊어질수록 기울기 소실(Vanishing Gradient) 문제가 발생해 학습이 어려워짐
    - ResNet은 출력값을 학습하는 대신, 입력과 출력 간의 차이(잔차)를 학습하도록 설계
    

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else 'cpu')

#### 하이퍼파라미터

In [13]:
EPOCHS     = 300
BATCH_SIZE = 128

#### 데이터셋 

In [14]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data/',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32, padding=4),
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data/',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)

100%|██████████| 170498071/170498071 [00:11<00:00, 14736479.60it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data/


#### ResNet 모델 만들기

In [15]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

#### 준비

In [16]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [17]:
model

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

#### 학습

In [18]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

#### 테스트

In [19]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

#### 코드 실행

In [ ]:
for epoch in tqdm( range(1, EPOCHS + 1) ):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')

  0%|          | 0/300 [00:00<?, ?it/s]

c:\Users\perso\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\torch\optim\lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


[1] Test Loss: 1.4262, Accuracy: 46.81%
[2] Test Loss: 1.1230, Accuracy: 60.13%
[3] Test Loss: 1.3840, Accuracy: 56.87%
[4] Test Loss: 0.8706, Accuracy: 70.68%
[5] Test Loss: 0.8685, Accuracy: 70.88%
[6] Test Loss: 0.9230, Accuracy: 69.26%
[7] Test Loss: 0.8097, Accuracy: 73.30%
[8] Test Loss: 0.9447, Accuracy: 69.96%
[9] Test Loss: 1.2459, Accuracy: 62.06%
[10] Test Loss: 0.6554, Accuracy: 78.17%
[11] Test Loss: 1.5906, Accuracy: 58.05%
[12] Test Loss: 0.7937, Accuracy: 74.25%
[13] Test Loss: 0.7358, Accuracy: 75.42%
[14] Test Loss: 1.0244, Accuracy: 67.35%
[15] Test Loss: 0.8184, Accuracy: 74.23%
[16] Test Loss: 0.7487, Accuracy: 75.80%
[17] Test Loss: 0.8143, Accuracy: 74.45%
[18] Test Loss: 1.0754, Accuracy: 68.10%
[19] Test Loss: 0.6112, Accuracy: 79.45%
[20] Test Loss: 0.6646, Accuracy: 78.45%
[21] Test Loss: 0.9261, Accuracy: 70.19%
[22] Test Loss: 0.8370, Accuracy: 71.09%
[23] Test Loss: 0.6629, Accuracy: 78.06%
[24] Test Loss: 0.8407, Accuracy: 73.45%
[25] Test Loss: 0.7140, A

- 엄청난 시간 소요. 예상은 1시간 30분 정도였으나, Idle time 등 문제로 13시간 44분 소요
- 정확도는 91.20 % 까지 올라감